In [ ]:
import pandas as pd
import numpy as np
from data_preprocessor import DataPreprocessor
from sklearn.model_selection import train_test_split

# настройка для красивого вывода
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 1. ПОДГОТОВКА ДАННЫХ
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

#  создаем проблему с пропусками
df['Trash_Column'] = np.nan
df.loc[0:10, 'Trash_Column'] = 1.0

# разбиваем на тренировочные и тестовые данные ДО фильтрации колонок
train, test = train_test_split(df, test_size=0.2, random_state=42)

print("=== ДАННЫЕ ДО ОБРАБОТКИ ===")
print(f"Размер таблицы: {train.shape[0]} строк, {train.shape[1]} колонок")
display(train.head(10))
print("-" * 50)

# 2. ОБРАБОТКА
# оставляем важные признаки + мусорную колонку для теста удаления
cols_to_use = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Cabin', 'Trash_Column']
train_subset = train[cols_to_use]
test_subset = test[cols_to_use] # готовим тест аналогично

preprocessor = DataPreprocessor(train_subset)

# обучение и трансформормация тренировочных данных
# используем std и median, как более устойчивые к выбросам методы (можно изменить)
train_p = preprocessor.fit_transform(threshold=0.5, method='std', strategy='median')

# применяем ПАРАМЕТРЫ ТРЕНИРОВКИ к тесту
test_p = preprocessor.transform(test_subset)

print("=== ДАННЫЕ ПОСЛЕ ОБРАБОТКИ ===")
print(f"Размер: {train_p.shape[0]} строк, {train_p.shape[1]} колонок")
display(train_p.head(10))
print("-" * 50)

# 3. ОТЧЕТ
print("=== ОТЧЕТ ===")

# список удаленных
removed = list(set(cols_to_use) - set(preprocessor.params['cols_to_keep']))
print(f"1. Удалено колонок (пропуски > 50%): {removed}")

# проверка заполнения пропусков
imputed = [col for col, val in preprocessor.params['impute_values'].items() if train_subset[col].isnull().any()]
print(f"2. Заполнены пропуски в: {imputed}")

# категории
print(f"3. Закодированы категории: {preprocessor.params['cat_columns']}")

# проверка нормализации
normalized = list(preprocessor.params['scaling_stats'].keys())
print(f"4. Нормализованы числовые признаки: {normalized}")

# 4. ПРОВЕРКА ТЕСТОВОЙ ВЫБОРКИ
print("\n=== ПРОВЕРКА ТЕСТОВОЙ ВЫБОРКИ ===")
print(f"Форма теста после трансформации: {test_p.shape}")
if test_p.isnull().sum().sum() == 0:
    print("В тестовых данных нет пропусков")
if list(train_p.columns) == list(test_p.columns):
    print("Структуры колонок трейна и теста идентичны")

=== ДАННЫЕ ДО ОБРАБОТКИ ===
Размер таблицы: 712 строк, 13 колонок


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Trash_Column
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S,NaN
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S,NaN
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S,NaN
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S,NaN
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S,NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C,NaN
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S,NaN
361,362,0,2,"del Carlo, Mr. Sebastiano",male,29.0,1,0,SC/PARIS 2167,27.7208,NaN,C,NaN
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S,NaN
55,56,1,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S,NaN


--------------------------------------------------
=== ДАННЫЕ ПОСЛЕ ОБРАБОТКИ ===
Размер: 712 строк, 7 колонок


,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
331,0,0.0,0.566474,0.055628,1,0,1
733,0,0.5,0.283740,0.025374,1,0,1
382,0,1.0,0.396833,0.015469,1,0,1
704,0,1.0,0.321438,0.015330,1,0,1
813,0,1.0,0.070118,0.061045,0,0,1
118,0,0.0,0.296306,0.483128,1,0,0
536,0,0.0,0.560191,0.051822,1,0,1
361,0,0.5,0.359135,0.054107,1,0,0
29,0,1.0,0.346569,0.015412,1,0,1
55,1,0.0,0.346569,0.069291,1,0,1


--------------------------------------------------
=== ОТЧЕТ ===
1. Удалено колонок (пропуски > 50%): ['Trash_Column', 'Cabin']
2. Заполнены пропуски в: ['Age', 'Embarked']
3. Закодированы категории: ['Sex', 'Embarked']
4. Нормализованы числовые признаки: ['Pclass', 'Age', 'Fare']

=== ПРОВЕРКА ТЕСТОВОЙ ВЫБОРКИ ===
Форма теста после трансформации: (179, 7)
В тестовых данных нет пропусков
Структуры колонок трейна и теста идентичны
